<a href="https://colab.research.google.com/github/LucasBorba1/project_hailmary/blob/main/project_hailmary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ================================
# CubeSat ADCS Simulator (Single File)
# ================================

import numpy as np
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt

# -------------------------------
# Reaction Wheel
# -------------------------------
class ReactionWheel:
    def __init__(self, axis, max_torque=0.01, inertia=1e-4):
        self.axis = np.array(axis) / np.linalg.norm(axis)
        self.max_torque = max_torque
        self.inertia = inertia
        self.omega = 0.0

    def apply_torque(self, torque_cmd, dt):
        torque = np.clip(torque_cmd, -self.max_torque, self.max_torque)
        self.omega += torque / self.inertia * dt
        return -torque * self.axis  # torque on CubeSat body

# -------------------------------
# CubeSat
# -------------------------------
class CubeSat:
    def __init__(self, mass=1.6, dimensions=(0.1,0.1,0.2), wheels=None):
        self.mass = mass
        self.dimensions = dimensions
        self.I = np.diag([0.5*mass*(dimensions[1]**2 + dimensions[2]**2),
                          0.5*mass*(dimensions[0]**2 + dimensions[2]**2),
                          0.5*mass*(dimensions[0]**2 + dimensions[1]**2)])
        self.I_inv = np.linalg.inv(self.I)

        self.omega = np.zeros(3)
        self.attitude = R.identity()
        self.wheels = wheels if wheels else []

        # history for plotting
        self.history = {'omega': [], 'attitude': []}

    def step(self, wheel_cmds, dt):
        total_torque = np.zeros(3)
        for wheel, cmd in zip(self.wheels, wheel_cmds):
            total_torque += wheel.apply_torque(cmd, dt)

        # Rigid body dynamics
        omega_dot = self.I_inv @ (total_torque - np.cross(self.omega, self.I @ self.omega))
        self.omega += omega_dot * dt

        # Integrate attitude
        rot_vec = self.omega * dt
        self.attitude = self.attitude * R.from_rotvec(rot_vec)

        # store history
        self.history['omega'].append(self.omega.copy())
        self.history['attitude'].append(self.attitude.as_quat().copy())

    def plot_omega(self):
        omega_hist = np.array(self.history['omega'])
        plt.figure(figsize=(6,4))
        plt.plot(omega_hist[:,0], label='wx')
        plt.plot(omega_hist[:,1], label='wy')
        plt.plot(omega_hist[:,2], label='wz')
        plt.legend()
        plt.xlabel('Step')
        plt.ylabel('Angular velocity [rad/s]')
        plt.title('CubeSat Body Angular Velocity')
        plt.show()

# -------------------------------
# Sensors
# -------------------------------
class Gyro:
    def __init__(self, noise_std=1e-4):
        self.noise_std = noise_std
    def read(self, omega_true):
        return omega_true + np.random.normal(0, self.noise_std, size=3)

class Accelerometer:
    def __init__(self, noise_std=1e-3):
        self.noise_std = noise_std
    def read(self, accel_true):
        return accel_true + np.random.normal(0, self.noise_std, size=3)

class Magnetometer:
    def __init__(self, noise_std=1e-5, magnetic_field=np.array([1.0,0.0,0.0])):
        self.noise_std = noise_std
        self.magnetic_field = np.array(magnetic_field)
    def read(self, attitude:R):
        mag_body = attitude.inv().apply(self.magnetic_field)
        return mag_body + np.random.normal(0, self.noise_std, size=3)

# -------------------------------
# Simple PID Controller
# -------------------------------
class PIDController:
    def __init__(self, Kp, Ki, Kd):
        self.Kp = Kp
        self.Ki = Ki
        self.Kd = Kd
        self.integral = np.zeros(3)
        self.prev_error = np.zeros(3)

    def step(self, error, dt):
        self.integral += error * dt
        derivative = (error - self.prev_error) / dt
        self.prev_error = error
        return self.Kp*error + self.Ki*self.integral + self.Kd*derivative

# -------------------------------
# Simulator
# -------------------------------
class Simulator:
    def __init__(self, cubesat, controller, sensors, dt=0.01):
        self.cubesat = cubesat
        self.controller = controller
        self.sensors = sensors
        self.dt = dt
        self.time = []

    def run(self, steps):
        for step in range(steps):
            # Read sensors
            gyro_reading = self.sensors['gyro'].read(self.cubesat.omega)
            accel_reading = self.sensors['accel'].read(np.zeros(3))
            mag_reading = self.sensors['mag'].read(self.cubesat.attitude)

            # Compute error (for demo, desired angular velocity = 0)
            error = -gyro_reading

            # Compute wheel torque commands
            wheel_cmds = self.controller.step(error, self.dt)

            # Step CubeSat dynamics
            self.cubesat.step(wheel_cmds, self.dt)
            self.time.append(step*self.dt)

# ================================
# Example usage
# ================================
# Create 4 pyramidal wheels
axes = np.array([[1,1,1],[1,-1,-1],[-1,1,-1],[-1,-1,1]])
axes = axes / np.linalg.norm(axes[0])
wheels = [ReactionWheel(axis) for axis in axes]

# CubeSat instance
sat = CubeSat(wheels=wheels)

# Sensors
sensors = {'gyro':Gyro(), 'accel':Accelerometer(), 'mag':Magnetometer()}

# Controller
pid = PIDController(Kp=0.05, Ki=0.01, Kd=0.001)

# Simulator
sim = Simulator(sat, pid, sensors, dt=0.1)
sim.run(steps=100)

# Plot results
sat.plot_omega()
